# Exploratory Data Analysis with Databricks

This example shows easily accessing data stored in Databricks using SQL Warehouses and all-purpose compute using credentials already provided in Workbench.

## Using SQL Warehouses

In [1]:
from databricks.sdk.core import Config
from databricks import sql
import pandas as pd
import os

config = Config(profile="workbench")

con = sql.connect(
  server_hostname=config.host,
  http_path="/sql/1.0/warehouses/ef4f89054a7fdcf3",
  access_token=config.token
)

In [5]:
query = """
        SELECT
            `member_id`,
            `zip_code`,
            `sub_grade`,
            `loan_amnt`,
            `term`,
            `int_rate`,
            `emp_title`,
            `emp_length`,
            `annual_inc`,
            `loan_status`,
            `title`,
            `out_prncp`
        FROM `sol_eng_demo_nickp`.`default`.`lending_club`
        WHERE
            `zip_code` = '996xx'
        ORDER BY sub_grade
    """

df = pd.read_sql(
    query,
    con
)
df

/tmp/ipykernel_537/419777910.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


,member_id,zip_code,sub_grade,loan_amnt,term,int_rate,emp_title,emp_length,annual_inc,loan_status,title,out_prncp
0,97645133,996xx,B3,23000,60 months,10.56%,None,n/a,48000,Current,Debt consolidation,22412.12
1,8326124,996xx,B1,27100,36 months,9.58%,Community health practitioner,4 years,63190,Current,Credit card refinancing,25789.27
2,78260753,996xx,C2,7200,36 months,14.03%,None,n/a,34200,Current,Debt consolidation,6708.27
3,96287631,996xx,D3,10000,60 months,19.42%,Transportation Coordinator,1 year,39000,Current,Debt consolidation,9695.43
4,21051028,996xx,D1,26000,60 months,17.47%,Loan Assistant,5 years,40000,Current,Debt consolidation,25165.23
...,...,...,...,...,...,...,...,...,...,...,...,...
63,98208525,996xx,D2,13000,60 months,18.45%,Maintenance,6 years,57480,Current,Credit card refinancing,12453.82
64,75720412,996xx,C4,10000,36 months,15.04%,Quality Control Manager,10+ years,87000,Current,Debt consolidation,9097.14
65,9217945,996xx,B4,24000,36 months,10.90%,Teacher,1 year,75000,Current,Major purchase,21702.53
66,6839056,996xx,C5,10000,36 months,16.01%,Head Cook,2 years,35600,Current,Credit card refinancing,8953.68


## Using Spark and All-Purpose Compute

In [2]:
from databricks.connect import DatabricksSession
from databricks.sdk.core import Config
import pandas as pd

config = Config(
    profile="workbench",
    cluster_id="0401-195338-1w0t2cgm",
    )

config

<Config: host=https://rstudio-partner-posit-default.cloud.databricks.com, token=***, profile=workbench, config_file=/tmp/trevor.nederlof/posit-workbench/d5de83c9ab5a784d15315/databricks.cfg, auth_type=pat, cluster_id=0401-195338-1w0t2cgm. Env: DATABRICKS_HOST, DATABRICKS_CONFIG_PROFILE, DATABRICKS_CONFIG_FILE>

In [3]:
spark = DatabricksSession.builder.sdkConfig(config).getOrCreate()
spark


In [6]:

query = """
        SELECT
            `member_id`,
            `zip_code`,
            `sub_grade`,
            `loan_amnt`,
            `term`,
            `int_rate`,
            `emp_title`,
            `emp_length`,
            `annual_inc`,
            `loan_status`,
            `title`,
            `out_prncp`
        FROM `sol_eng_demo_nickp`.`default`.`lending_club`
        WHERE
            `zip_code` = '996xx'
        ORDER BY sub_grade
    """

df = spark.sql(query).toPandas()
df

,member_id,zip_code,sub_grade,loan_amnt,term,int_rate,emp_title,emp_length,annual_inc,loan_status,title,out_prncp
0,82754316,996xx,A1,40000,60 months,5.31%,Nurse,1 year,170000,Current,Debt consolidation,37650.43
1,22507054,996xx,A1,23800,36 months,5.31%,Equipment Manager,10+ years,166000,Current,Home improvement,20716.25
2,19080634,996xx,A1,40000,60 months,5.31%,Journeyman utilityman,10+ years,90000,Current,Debt consolidation,38241.59
3,664208,996xx,A2,40000,36 months,6.07%,owner,8 years,70000,Current,Major purchase,35905.8
4,31133216,996xx,A2,6000,36 months,6.19%,Voice Over Artist,2 years,20000,Current,Credit card refinancing,5541.34
...,...,...,...,...,...,...,...,...,...,...,...,...
63,72130305,996xx,D3,24000,60 months,19.42%,CDL driver/sales representative,2 years,65000,Current,Debt consolidation,23017.51
64,76489258,996xx,D3,12000,60 months,19.42%,Equipment Operator,10+ years,73000,Current,Debt consolidation,11380.93
65,96287631,996xx,D3,10000,60 months,19.42%,Transportation Coordinator,1 year,39000,Current,Debt consolidation,9695.43
66,34268470,996xx,D4,29525,36 months,20.39%,vice president,10+ years,150000,Current,Debt consolidation,26413.74
